<a href="https://colab.research.google.com/github/carolynw898/STAT946Proj/blob/main/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from utils import lossFunc, relativeErr
import torch
from torch.utils.data import DataLoader

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
n_embd = 512
timesteps = 1000
batch_size = 1
learning_rate = 1e-4
num_epochs = 10
blockSize = 32
testBlockSize = 400
numVars = 1
numYs = 1
numPoints = 250
target = 'Skeleton'
const_range = [-2.1, 2.1]
trainRange = [-3.0, 3.0]
decimals = 8
addVars = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import numpy as np
import glob
from utils import processDataFiles, CharDataset, tokenize_equation
import random
import json

# files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_train.json")
files = glob.glob("0_1_0_13062021_174033.json")
text = processDataFiles(files)
text = text.split('\n') # convert the raw text to a set of examples
# skeletons = []
skeletons = [json.loads(item)['Skeleton'] for item in text if item.strip()]
all_tokens = set()
for eq in skeletons:
    all_tokens.update(tokenize_equation(eq))
integers = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
all_tokens.update(integers)  # add all integers to the token set
tokens = sorted(list(all_tokens) + ['_', 'T', '<', '>', ':'])  # special tokens
trainText = text[:-1] if len(text[-1]) == 0 else text
random.shuffle(trainText) # shuffle the dataset, it's important specailly for the combined number of variables experiment
train_dataset = CharDataset(trainText, blockSize, tokens=all_tokens, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, target=target, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

data has 966 examples, 19 unique.


In [16]:
integers = {'0', '1', '2', '3', '4', '5', '6', '7', '8', '9'}
all_tokens.update(integers)  # add all integers to the token set

In [ ]:
print(all_tokens)

{'7', 'log', '2', '+', 'exp', '5', 'C', 'x1', '8', '9', '1', '0', '6', '**', '3', 'cos', '*', '4', '(', ')', '/', 'sin'}


In [18]:
tokens

['(',
 ')',
 '*',
 '**',
 '+',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 ':',
 '<',
 '>',
 'C',
 'T',
 '_',
 'cos',
 'exp',
 'log',
 'sin',
 'x1']

In [6]:
print(skeletons[:10])

['C*exp(C*x1/(C*x1))+C', 'C*exp(C*x1)*C*cos(C*x1**2+C*x1)+C', 'C*x1**5+C*x1**4+C*x1**3+C*x1**2*C*log(C*x1)+C*x1**2+C*x1+C', 'C*x1/(C*x1+C)+C', 'C*x1*C*log(C*x1**4+C*x1**3+C*x1**2+C*x1+C)*C*sin(C*x1)+C', 'C*x1**2+C*x1+C', 'C*x1+C*sin(C*x1)+C', 'C*x1+C/x1+C', 'C*x1**5+C*x1**4+C*x1**3+C*x1**2+C*x1+C', 'C*x1**4+C*x1**2+C']


In [30]:
idx = np.random.randint(train_dataset.__len__())
inputs, outputs, points, variables = train_dataset.__getitem__(idx)
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

id:316674
outputs:C*sin(C*cos(C*x1+C))+C>_________
variables:1


In [31]:
files = glob.glob("/content/drive/MyDrive/Colab Notebooks/STAT946_proj/data/1_var_test.json")
textTest = processDataFiles(files)
textTest = textTest.split('\n') # convert the raw text to a set of examples
# test_dataset_target = CharDataset(textTest, blockSize, chars, target=target)
test_dataset = CharDataset(textTest, testBlockSize, chars, numVars=numVars,
                        numYs=numYs, numPoints=numPoints, addVars=addVars,
                        const_range=const_range, xRange=trainRange, decimals=decimals)

idx = np.random.randint(test_dataset.__len__())
inputs, outputs, points, variables = test_dataset.__getitem__(idx)
print(points.min(), points.max())
inputs = ''.join([train_dataset.itos[int(i)] for i in inputs])
outputs = ''.join([train_dataset.itos[int(i)] for i in outputs])
print('id:{}\noutputs:{}\nvariables:{}'.format(idx,outputs,variables))

test_loader = DataLoader(
        test_dataset,
        batch_size=batch_size,
        pin_memory=True,
        shuffle=False,
        num_workers=2
    )

data has 967 examples, 49 unique.
tensor(-2.9041) tensor(2.9033)
id:57
outputs:0.40770798158663046*sin(1.246840487111979*log(2.052510259517575*x1)+-0.7785508700595822)+1.7307411861774487>____________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________________
variables:1


In [32]:
import torch
import torch.nn as nn
from torch.nn import functional as F
from typing import Tuple


# from SymbolicGPT: https://github.com/mojivalipour/symbolicgpt/blob/master/models.py
class PointNetConfig:
    """base PointNet config"""

    def __init__(
        self,
        embeddingSize,
        numberofPoints,
        numberofVars,
        numberofYs,
        method="GPT",
        varibleEmbedding="NOT_VAR",
        **kwargs,
    ):
        self.embeddingSize = embeddingSize
        self.numberofPoints = numberofPoints  # number of points
        self.numberofVars = numberofVars  # input dimension (Xs)
        self.numberofYs = numberofYs  # output dimension (Ys)
        self.method = method
        self.varibleEmbedding = varibleEmbedding

        for k, v in kwargs.items():
            setattr(self, k, v)


class tNet(nn.Module):
    """
    The PointNet structure in the orginal PointNet paper:
    PointNet: Deep Learning on Point Sets for 3D Classification and Segmentation by Qi et. al. 2017
    """

    def __init__(self, config):
        super(tNet, self).__init__()

        self.activation_func = F.relu
        self.num_units = config.embeddingSize

        self.conv1 = nn.Conv1d(
            config.numberofVars + config.numberofYs, self.num_units, 1
        )
        self.conv2 = nn.Conv1d(self.num_units, 2 * self.num_units, 1)
        self.conv3 = nn.Conv1d(2 * self.num_units, 4 * self.num_units, 1)
        self.fc1 = nn.Linear(4 * self.num_units, 2 * self.num_units)
        self.fc2 = nn.Linear(2 * self.num_units, self.num_units)

        # self.relu = nn.ReLU()

        self.input_batch_norm = nn.BatchNorm1d(config.numberofVars + config.numberofYs)
        # self.input_layer_norm = nn.LayerNorm(config.numberofPoints)

        self.bn1 = nn.BatchNorm1d(self.num_units)
        self.bn2 = nn.BatchNorm1d(2 * self.num_units)
        self.bn3 = nn.BatchNorm1d(4 * self.num_units)
        self.bn4 = nn.BatchNorm1d(2 * self.num_units)
        self.bn5 = nn.BatchNorm1d(self.num_units)

    def forward(self, x):
        """
        :param x: [batch, #features, #points]
        :return:
            logit: [batch, embedding_size]
        """
        x = self.input_batch_norm(x)
        x = self.activation_func(self.bn1(self.conv1(x)))
        x = self.activation_func(self.bn2(self.conv2(x)))
        x = self.activation_func(self.bn3(self.conv3(x)))
        x, _ = torch.max(x, dim=2)  # global max pooling
        assert x.size(1) == 4 * self.num_units

        x = self.activation_func(self.bn4(self.fc1(x)))
        x = self.activation_func(self.bn5(self.fc2(x)))
        # x = self.fc2(x)

        return x


class NoisePredictionTransformer(nn.Module):
    """Predicts continuous noise in the embedding space for diffusion-based symbolic regression."""

    def __init__(
        self,
        vocab_size: int,
        max_seq_len: int,
        padding_idx: int = 0,
        n_layer: int = 6,
        n_head: int = 8,
        n_embd: int = 512,
        max_timesteps: int = 1000,
    ):
        super().__init__()
        # self.tok_emb = nn.Embedding(
        #     vocab_size, n_embd, padding_idx=torch.tensor(padding_idx, dtype=torch.long)
        # )
        self.pos_emb = nn.Parameter(torch.zeros(1, max_seq_len, n_embd))
        self.time_emb = nn.Embedding(max_timesteps, n_embd)

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=n_embd,
            nhead=n_head,
            dim_feedforward=n_embd * 4,
            activation="gelu",
            batch_first=True,
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layer)
        self.cross_attention = nn.MultiheadAttention(n_embd, n_head, batch_first=True)

    def forward(
        self,
        x_t: torch.Tensor,
        t: torch.Tensor,
        condition: torch.Tensor,
    ) -> torch.Tensor:
        """Predicts noise from noisy token indices, timestep, and condition.

        Args:
            x_t: [B, L] - Noisy expression at time t (token indices)
            t: [B] - Timestep
            condition: [B, n_embd] - Combined T-Net and variable embeddings

        Returns:
            noise_pred: [B, L, n_embd] - Predicted noise in embedding space
        """
        # print(x_t.shape)
        _, L, _ = x_t.shape

        # print(f"x_t min: {x_t.min().item()}, max: {x_t.max().item()}")
        # print(self.tok_emb.weight.shape[0])  # Number of embeddings

        # tok_emb = self.tok_emb(x_t.long())
        pos_emb = self.pos_emb[:, :L, :]
        time_emb = self.time_emb(t).unsqueeze(1)
        x = x_t + pos_emb + time_emb

        condition = condition.unsqueeze(1)
        attn_output, _ = self.cross_attention(x, condition, condition)
        x = x + attn_output  # Residual connection

        noise_pred = self.encoder(x)
        return noise_pred


class SymbolicDiffusion(nn.Module):
    def __init__(
        self,
        pconfig,
        vocab_size: int,
        max_seq_len: int,
        padding_idx: int = 0,
        max_num_vars: int = 9,
        n_layer: int = 4,
        n_head: int = 4,
        n_embd: int = 512,
        timesteps: int = 1000,
        beta_start: float = 0.0001,
        beta_end: float = 0.02,
        tok_emb_weights: torch.Tensor = None,
        vars_emb_weights: torch.Tensor = None,
        train_decoder: bool = True,
    ):
        super().__init__()
        self.timesteps = timesteps
        self.n_embd = n_embd
        self.max_seq_len = max_seq_len
        self.vocab_size = vocab_size
        self.padding_idx = padding_idx
        self.tok_emb_weights = tok_emb_weights
        self.vars_emb_weights = vars_emb_weights

        # Initialize embedding layers
        self.tok_emb = nn.Embedding(vocab_size, n_embd, padding_idx=padding_idx)
        self.vars_emb = nn.Embedding(max_num_vars, n_embd)

        # Load and freeze (requires_grad = False ensures they won't be updated) weights if provided
        if tok_emb_weights is not None:
            self.tok_emb.weight = nn.Parameter(tok_emb_weights, requires_grad=False)

        if vars_emb_weights is not None:
            self.vars_emb.weight = nn.Parameter(vars_emb_weights, requires_grad=False)

        self.tnet = tNet(pconfig)
        self.transformer = NoisePredictionTransformer(
            vocab_size,
            max_seq_len,
            padding_idx,
            n_layer=n_layer,
            n_head=n_head,
            n_embd=n_embd,
            max_timesteps=timesteps,
        )

        self.train_decoder = train_decoder
        if train_decoder:
            self.decoder = nn.Linear(n_embd, vocab_size)
        else:
            self.decoder = self.round

        self.register_buffer("beta", torch.linspace(beta_start, beta_end, timesteps))
        self.register_buffer("alpha", 1.0 - self.beta)
        self.register_buffer("alpha_bar", torch.cumprod(self.alpha, dim=0))

    def xtoi(self, xt):
        B, L, _ = xt.shape
        emb_table = self.tok_emb.weight  # [vocab_size, n_embd]

        xt_flat = xt.view(B * L, -1)  # [B*L, n_embd]
        # Compute squared L2 distance: ||xt - emb_table||^2
        distances = torch.cdist(xt_flat, emb_table, p=2).pow(2)  # [B*L, vocab_size]
        nearest_indices = torch.argmin(distances, dim=1).view(B, L)  # [B, L]

        return nearest_indices

    def round(self, xt):
        emb_table = self.tok_emb.weight
        idx = self.xtoi(xt)
        return emb_table[idx]

    def q_sample(
        self,
        x0: torch.Tensor,
        t: torch.Tensor,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """Adds Gaussian noise to the input embeddings at time t.

        Args:
            x0: [B, L, n_embd] - Original expression embeddings
            t: [B] - Timestep

        Returns:
            xt: [B, L, n_embd] - Noisy embeddings at time t
            noise: [B, L, n_embd] - Noise added to the original embeddings
        """
        noise = torch.randn_like(x0, dtype=torch.float)
        sqrt_alpha_bar = torch.sqrt(self.alpha_bar[t]).view(-1, 1, 1)
        sqrt_subone_alpha_bar = torch.sqrt(1 - self.alpha_bar[t]).view(-1, 1, 1)
        xt = sqrt_alpha_bar * x0 + sqrt_subone_alpha_bar * noise
        return xt, noise

    def p_sample(
        self,
        x: torch.Tensor,
        t: int,
        noise_pred: torch.Tensor,
    ) -> torch.Tensor:
        """Denoises the noisy embeddings at time t.

        Args:
            x: [B, L, n_embd] - Noisy embeddings at time t
            t: int - Current timestep (scalar)
            condition: [B, n_embd] - Combined T-Net and variable embeddings
            device: str - Device to use (e.g., "cuda")

        Returns:
            x: [B, L, n_embd] - Denoised embeddings
        """
        B = x.shape[0]
        beta_t = self.beta[t].view(-1, 1, 1)
        alpha_t = self.alpha[t].view(-1, 1, 1)
        alpha_bar_t = self.alpha_bar[t].view(-1, 1, 1)
        mean = (1 / torch.sqrt(alpha_t)) * (
            x - (beta_t / torch.sqrt(1 - alpha_bar_t)) * noise_pred
        )
        return mean

    def forward(
        self,
        points: torch.Tensor,
        tokens: torch.Tensor,
        variables: torch.Tensor,
        t: torch.Tensor,
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
        """Training forward pass to predict noise added to embeddings.

        Args:
            points: [B, 2, 250] - Dataset points for conditioning
            tokens: [B, L] - Ground truth expression (token indices)
            variables: [B] - Number of variables per sample
            t: [B] - Timestep

        Returns:
            y_pred: [B, L] - Generated expression (token indices)
            noise_pred: [B, L, n_embd] - Predicted noise
            noise: [B, L, n_embd] - Actual noise added
        """
        B = tokens.shape[0]

        condition = self.tnet(points)
        vars_emb = self.vars_emb(variables)
        condition = condition + vars_emb

        token_emb = self.tok_emb(tokens)
        xt, noise = self.q_sample(token_emb, t)
        noise_pred = self.transformer(xt, t, condition)
        y_pred = self.p_sample(xt, t, noise_pred)
        if self.train_decoder:
            y_pred = self.decoder(y_pred)
        return y_pred, noise_pred, noise

    @torch.no_grad()
    def sample(
        self,
        points: torch.Tensor,
        variables: torch.Tensor,
        device: str = "cuda",
    ) -> torch.Tensor:
        """Generates a sample by denoising from random noise.

        Args:
            points: [B, 2, 250] - Dataset points for conditioning
            variables: [B] - Number of variables per sample
            device: str - Device to use (e.g., "cuda")

        Returns:
            xt: [B, L] - Generated expression (token indices)
        """
        condition = self.tnet(points) + self.vars_emb(variables)  # [B, n_embd]
        B = condition.shape[0]

        xt_emb = torch.randn(B, self.max_seq_len, self.n_embd, device=device)
        t_tensor = torch.zeros(B, dtype=torch.long, device=device)  # [B]
        for t in range(self.timesteps - 1, -1, -1):
            t_tensor.fill_(t)  # Update in-place: [B] all set to t
            noise_pred = self.transformer(xt_emb, t_tensor, condition)
            xt_emb = self.p_sample(xt_emb, t, noise_pred)  # t as int for p_sample
            if self.train_decoder:
                xt_logits = self.decoder(xt_emb)
                xt = torch.argmax(xt_logits, dim=-1)
                xt_emb = self.tok_emb(xt)
            else:
                xt_emb = self.round(xt_emb)

        xt = self.xtoi(xt_emb)
        return xt

    def loss_fn(
        self,
        noise_pred: torch.Tensor,  # [B, L, n_embd]
        noise: torch.Tensor,  # [B, L, n_embd]
        pred_logits: torch.Tensor,  # [B, L, vocab_size] or [B, L, n_embd]
        tokens: torch.Tensor,  # [B, L]
        t: torch.Tensor,  # [B]
        verbose: bool = False,
    ) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor, torch.Tensor]:
        B, L = tokens.shape
        device = tokens.device

        mse_loss = F.mse_loss(noise_pred, noise)
        ce_weight = 1.0 - (t.float() / self.timesteps)

        if self.train_decoder:
            ce_loss = F.cross_entropy(
                pred_logits.view(-1, pred_logits.size(-1)),
                tokens.view(-1),
                reduction="none",
                ignore_index=self.padding_idx,
            ).view(B, L)
            weighted_ce_loss = (ce_weight.unsqueeze(1) * ce_loss).mean()
            rounding_loss = torch.tensor(0.0, device=device)
        else:
            weighted_ce_loss = torch.tensor(0.0, device=device)
            rounding_loss = F.mse_loss(pred_logits, self.tok_emb(tokens))  # predict exact target embeddings

        total_loss = mse_loss + weighted_ce_loss + rounding_loss

        if verbose:
            print(f"MSE: {mse_loss.item():.4f}, CE: {weighted_ce_loss.item():.4f}, Rounding: {rounding_loss.item():.4f}")

        return total_loss, mse_loss, weighted_ce_loss, rounding_loss


In [33]:
import torch
import json
from scipy.optimize import minimize
from math import log

@torch.no_grad()
def test_model(model, test_loader, textTest, train_dataset, device):
    results = {'target': [], 'predicted_skeleton' : [], 'predicted': [], 'error': []}

    for batch_idx, (_, tokens, points, variables) in enumerate(test_loader):
        points = points.to(device)    # [B, 2, 250]
        tokens = tokens.to(device)    # [B, L]
        variables = variables.to(device)  # [B]

        generated_tokens = model.sample(points, variables, device)

        for i in range(batch_size):
            # Ground truth
            eq = ''.join([train_dataset.itos[int(i)] for i in tokens[0]])
            eq = eq.strip(train_dataset.paddingToken).split('>')
            eq = eq[0] #if len(eq[0])>=1 else eq[1]
            eq = eq.strip('<').strip(">")

            # Predicted
            predicted_tokens = generated_tokens[i].cpu().numpy()
            predicted = ''.join([train_dataset.itos[int(idx)] for idx in predicted_tokens])
            predicted = predicted.strip(train_dataset.paddingToken).split('>')
            predicted = predicted[0] if len(predicted[0]) >= 1 else predicted[1]
            predicted = predicted.strip('<').strip(">")
            predicted = predicted.replace('Ce', 'C*e')
            predicted_skeleton = predicted


            t = json.loads(textTest[i])
            # train a regressor to find the constants (too slow)
            c = [1.0 for i,x in enumerate(predicted) if x=='C'] # initialize coefficients as 1
            # c[-1] = 0 # initialize the constant as zero
            b = [(-2,2) for i,x in enumerate(predicted) if x=='C']  # bounds on variables
            try:
                if len(c) != 0:
                    # This is the bottleneck in our algorithm
                    # for easier comparison, we are using minimize package
                    cHat = minimize(lossFunc, c, #bounds=b,
                                args=(predicted, t['X'], t['Y']))

                    predicted = predicted.replace('C','{}').format(*cHat.x)
            except ValueError:
                raise 'Err: Wrong Equation {}'.format(predicted)
            except Exception as e:
                raise 'Err: Wrong Equation {}, Err: {}'.format(predicted, e)

            Ys = [] #t['YT']
            Yhats = []
            for xs in t['XT']:
              try:
                  eqTmp = eq + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  YEval = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"TA: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  YEval = 100  # Assign a default value or handle as needed
                  #raise
                  YEval = 100
              Ys.append(YEval)

              try:
                  eqTmp = predicted + ''  # copy eq
                  eqTmp = eqTmp.replace(' ', '')
                  eqTmp = eqTmp.replace('\n', '')
                  for i, x in enumerate(xs):
                      # replace xi with the value in the eq
                      eqTmp = eqTmp.replace('x{}'.format(i + 1), str(x))
                      if ',' in eqTmp:
                          assert 'There is a "," in the equation!'
                  Yhat = eval(eqTmp)
              except Exception as e:
                  continue
                  print(f"PR: Invalid equation encountered. Eq: {eqTmp}, Reason: {e}")
                  Yhat = 100  # Assign a default value or handle as needed
              Yhats.append(Yhat)
            err = relativeErr(Ys,Yhats, info=True)


            results['target'].append(eq)
            results['predicted_skeleton'].append(predicted_skeleton)
            results['predicted'].append(predicted)
            results['error'].append(err)

            print(f"\nSample {batch_idx * batch_size + i + 1}:")
            print(f"Target: {eq}")
            print(f"Predicted Skeleton: {predicted_skeleton}")
            print(f"Predicted: {predicted}")
            print(f"Relative Error: {err:.6f}")
            print("-" * 50)

    return results

In [34]:
pconfig = PointNetConfig(
    embeddingSize=n_embd,
    numberofPoints=250,
    numberofVars=1,
    numberofYs=1,
)
model = SymbolicDiffusion(
    pconfig=pconfig,
    vocab_size=train_dataset.vocab_size,
    max_seq_len=32,
    padding_idx=test_dataset.paddingID,
    max_num_vars=9,
    n_layer=8,
    n_head=8,
    n_embd=n_embd,
    timesteps=timesteps,
    beta_start=0.0001,
    beta_end=0.02,
).to(device)


print(train_dataset.itos)

model_path = "/content/drive/MyDrive/Colab Notebooks/STAT946_proj/models/best_model_94.pth"
model.load_state_dict(torch.load(model_path, map_location=device))
model.eval()

print("Testing SymbolicDiffusion model...")
test_results = test_model(model, test_loader, textTest, train_dataset, device)

print("\nSummary:")
for i in range(len(test_results['target'])):
    print(f"Sample {i+1}:")
    print(f"  Target: {test_results['target'][i]}")
    print(f"  Predicted Skeleton: {test_results['predicted_skeleton'][i]}")
    print(f"  Predicted: {test_results['predicted'][i]}")
    print(f"  Error: {test_results['error'][i]:.6f}")

{0: '\n', 1: ' ', 2: '"', 3: '(', 4: ')', 5: '*', 6: '+', 7: ',', 8: '-', 9: '.', 10: '/', 11: '0', 12: '1', 13: '2', 14: '3', 15: '4', 16: '5', 17: '6', 18: '7', 19: '8', 20: '9', 21: ':', 22: ':', 23: '<', 24: '>', 25: 'C', 26: 'E', 27: 'Q', 28: 'S', 29: 'T', 30: 'X', 31: 'Y', 32: '[', 33: ']', 34: '_', 35: 'c', 36: 'e', 37: 'g', 38: 'i', 39: 'k', 40: 'l', 41: 'n', 42: 'o', 43: 'p', 44: 's', 45: 't', 46: 'x', 47: '{', 48: '}'}
Testing SymbolicDiffusion model...
Exception has been occured! EQ: s*xs)1.0x+*1*1.0**(1.0*1.47051444.0x*i1(1.01.01.01.47051444.0xo+, OR: s*xs)1.0x+*1*1.0**(1.0*x1.0x*i1(1.01.01.0x1.0xo+
Exception has been occured! EQ: s*xs)1.0x+*1*1.0**(1.0*-1.21969295.0x*i1(1.01.01.0-1.21969295.0xo+, OR: s*xs)1.0x+*1*1.0**(1.0*x1.0x*i1(1.01.01.0x1.0xo+
Exception has been occured! EQ: s*xs)1.0x+*1*1.0**(1.0*-0.09552825.0x*i1(1.01.01.0-0.09552825.0xo+, OR: s*xs)1.0x+*1*1.0**(1.0*x1.0x*i1(1.01.01.0x1.0xo+
Exception has been occured! EQ: s*xs)1.0x+*1*1.0**(1.0*-1.40721835.0x*i1(1.

KeyboardInterrupt: 